# Trabajo Practico N°1 NLP
Integrantes: 
1. Rodriguez y Barros, Francisco : 
2. Slepoy, David : S-5782/7

### Importamos las librerias

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

In [11]:
juegos_df = pd.read_csv('bgg_database.csv')  # Asegúrate de que este archivo esté en el mismo directorio
peliculas_df = pd.read_csv('IMDB-Movie-Data.csv')  # Asegúrate de que este archivo esté en el mismo directorio
libros_df = pd.read_csv('df_libros.csv')  # Este debe ser el resultado de tu web scraping

Elegimos para poder determinar el estado de animo del usuario. la Similitud del Coseno probablemente por su capacidad para trabajar con representaciones vectoriales del texto, ya que esta nos permitirá captar las relaciones semánticas entre el usuario y lo que nos ingrese con los diferentes estados de ánimo.

Creamos el modelo de regresion logistica

In [12]:
spanish_stop_words = stopwords.words('spanish')

In [13]:
# Definición de las etiquetas
labels = {
    0: "Melancólico",
    1: "Alegre",
    2: "Ni fu ni fa"
}

# Dataset de ejemplo
dataset = [
    # Ejemplos de "Alegre"
    (1, "¡Hoy es un día soleado y me siento lleno de energía!"),
    (1, "Estoy emocionado porque voy a ver a mis amigos esta noche."),
    (1, "La música que escucho me hace sentir muy feliz."),
    (1, "Acabo de recibir buenas noticias, ¡qué alegría!"),
    (1, "Me encanta pasear por el parque en primavera."),
    (1, "El chocolate siempre me alegra el día."),
    (1, "La sonrisa de un niño puede iluminar cualquier día."),
    (1, "¡No puedo esperar para ir a la fiesta!"),
    (1, "Este libro es tan divertido que no puedo dejar de reír."),
    (1, "La compañía de mi familia siempre me hace sentir bien."),

    # Ejemplos de "Melancólico"
    (0, "A veces me siento nostálgico al recordar viejos tiempos."),
    (0, "El clima gris de hoy refleja mi estado de ánimo."),
    (0, "Me siento triste porque extraño a un ser querido."),
    (0, "Las lágrimas caen sin que pueda controlarlas."),
    (0, "La soledad puede ser muy abrumadora a veces."),
    (0, "Recuerdo ese lugar feliz, pero ya no puedo volver."),
    (0, "Hoy es uno de esos días en los que no tengo ganas de hacer nada."),
    (0, "La canción que escucho me trae recuerdos tristes."),
    (0, "La pérdida de algo querido siempre deja un vacío."),
    (0, "Siento que todo está en silencio, como si el mundo me ignorara."),

    # Ejemplos de "Ni fu ni fa"
    (2, "Hoy ha sido un día normal, sin nada emocionante."),
    (2, "Me siento indiferente ante las noticias que escuché."),
    (2, "No estoy ni feliz ni triste, simplemente estoy."),
    (2, "La película que vi no fue buena ni mala, solo estaba ahí."),
    (2, "El clima es aceptable, pero no me afecta mucho."),
    (2, "A veces no sé qué pensar sobre ciertas situaciones."),
    (2, "No tengo emociones fuertes sobre el evento que se viene."),
    (2, "La comida estaba bien, pero no me dejó ninguna impresión."),
    (2, "Este libro es un poco aburrido, pero lo seguiré leyendo."),
    (2, "Estoy aquí, pero no tengo ganas de hacer nada en especial.")
]


# Preparar X e y
X = [text for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización de los textos con eliminación de palabras vacías
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_vectorized = vectorizer.fit_transform(X_train)  # Ajusta y transforma el conjunto de entrenamiento
X_test_vectorized = vectorizer.transform(X_test)  # Solo transforma el conjunto de prueba

# Creación y entrenamiento del modelo de Regresión Logística
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.5
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.33      0.50      0.40         2
           2       0.67      1.00      0.80         2

    accuracy                           0.50         6
   macro avg       0.67      0.50      0.40         6
weighted avg       0.67      0.50      0.40         6



c:\Users\david\OneDrive\Escritorio\T.U.I.A\4.2 PROCESAMIENTO DEL LENGUAJE NATURAL\directorio\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [14]:
def clasificar_estado_animo(frase_usuario):
    """
    Clasifica el estado de ánimo de una frase ingresada por el usuario.

    Args:
        frase_usuario (str): La frase que describe el estado de ánimo.

    Returns:
        str: El estado de ánimo clasificado ("Melancólico", "Alegre", "Ni fu ni fa").
    """
    # Vectorizar la nueva frase
    frase_vectorizada = vectorizer.transform([frase_usuario.lower()])  # Asegúrate de que el vectorizador esté previamente ajustado
    # Predecir la etiqueta
    etiqueta_predicha = modelo_LR.predict(frase_vectorizada)
    return labels[etiqueta_predicha[0]]  # Retornar la etiqueta correspondiente


In [15]:
# Función para recomendar basado en el estado de ánimo
def recomendar(frase_usuario, estado_animo):
    juegos_descripcion = juegos_df['description'].fillna('').tolist()
    peliculas_descripcion = peliculas_df['Description'].fillna('').tolist()
    libros_descripcion = libros_df['Summary'].fillna('').tolist()
    # Vectorizar la frase ingresada por el usuario
    frase_vectorizada = vectorizer.transform([frase_usuario.lower()])  # Solo vectorizar la frase del usuario

    # Vectorizamos las descripciones también
    juegos_vectorizados = vectorizer.transform(juegos_descripcion)
    peliculas_vectorizados = vectorizer.transform(peliculas_descripcion)
    libros_vectorizados = vectorizer.transform(libros_descripcion)

    # Calcular similitud
    similitudes_juegos = cosine_similarity(frase_vectorizada, juegos_vectorizados)
    similitudes_peliculas = cosine_similarity(frase_vectorizada, peliculas_vectorizados)
    similitudes_libros = cosine_similarity(frase_vectorizada, libros_vectorizados)

    # Obtener las recomendaciones basadas en similitud
    if estado_animo == "Alegre":
        top_juegos = juegos_df.iloc[similitudes_juegos.argsort()[0][-3:]]  # Top 3 juegos
        top_peliculas = peliculas_df.iloc[similitudes_peliculas.argsort()[0][-3:]]  # Top 3 películas
        return top_juegos, top_peliculas

    elif estado_animo == "Melancólico":
        top_libros = libros_df.iloc[similitudes_libros.argsort()[0][-3:]]  # Top 3 libros
        return top_libros,

    else:  # Ni fu ni fa
        top_juegos = juegos_df.iloc[similitudes_juegos.argsort()[0][-3:]]
        top_libros = libros_df.iloc[similitudes_libros.argsort()[0][-3:]]
        return top_juegos, top_libros

In [16]:
# Ingreso de preferencias del usuario
frase_usuario = input("Ingresa una frase que describa tu estado de ánimo: ")
estado_animo = clasificar_estado_animo(frase_usuario)
print(f"Tu estado de ánimo es: {estado_animo}")

# Obtener recomendaciones
recomendaciones = recomendar(frase_usuario, estado_animo)
# Imprimir recomendaciones
if estado_animo == "Alegre":
    print("Recomendaciones de Juegos:")
    print(recomendaciones[0])
    print("Recomendaciones de Películas:")
    print(recomendaciones[1])
elif estado_animo == "Melancólico":
    print("Recomendaciones de Libros:")
    print(recomendaciones[0])
else:
    print("Recomendaciones de Juegos:")
    print(recomendaciones[0])
    print("Recomendaciones de Libros:")
    print(recomendaciones[1])


Tu estado de ánimo es: Ni fu ni fa
Recomendaciones de Juegos:
     rank                                        game_name  \
998   999                                          Qwirkle   
999  1000  Dungeons & Dragons: Castle Ravenloft Board Game   
0       1                                Brass: Birmingham   

                                             game_href  geek_rating  \
998  https://boardgamegeek.com/boardgame/25669/qwirkle        6.616   
999  https://boardgamegeek.com/boardgame/59946/dung...        6.615   
0    https://boardgamegeek.com/boardgame/224517/bra...        8.415   

     avg_rating  num_voters  \
998        6.77     20334.0   
999        6.98      9290.0   
0          8.60     46836.0   

                                           description  yearpublished  \
998  The abstract game of Qwirkle consists of 108 w...           2006   
999  Castle Ravenloft Boardgame by Bill Slavicsek, ...           2010   
0    Brass: Birmingham is an economic strategy game...      